# Predictions from new data
---

In [1]:
# ignore warnings
import warnings
warnings.filterwarnings("ignore")

# pandas, numpy, matplotlib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Keras
from keras.models import Sequential, load_model
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor

# sci-kit learn
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.externals import joblib

Using TensorFlow backend.


## Data Setup

In [9]:
# get weather data
weather_data = pd.read_csv('resources/weather_forcast_5day.csv')

# add in column for daylight (seconds)   'dl_sec'
weather_data['dl_sec'] = 43200   # temporary value (12 hours)    FIX THIS!!

# relevant columns
features = [
    'clouds_all', 'temp_f', 'pressure', 'humidity', 'wind_speed', 'wind_deg', 'hour', 'day_of_year',
    'month', 'sin_day', 'cos_day', 'sin_hour', 'cos_hour', 'sin_month', 'cos_month', 'dl_sec'
]

# set parameters to test
forcast_data = weather_data[features].copy()

## Model

In [10]:
def deep_model():
    
    '''Build model with 2 hidden layers'''
    
    model = Sequential()
    model.add(Dense(units=input_dim * 4, activation='relu', input_dim=input_dim))
    model.add(Dense(units=input_dim * 2, activation='relu'))
    model.add(Dense(units=input_dim, activation='relu'))
    model.add(Dense(units=1))  # 1 output
    
    # compile with mean-squared error loss function
    model.compile(loss='mean_squared_error',
                 optimizer='adam',
                 metrics=['mse'])
    return model

## Predictions

In [21]:
# blank dict for predictions
predictions = {}

# load in pipeline
pipeline = joblib.load('nn_reg_pipeline.pkl')

# set model for pipeline as mpls neural net regression model
pipeline.named_steps['estimator'].model = load_model('mpls_nn_reg.h5')

# store predictions
predictions['mpls'] = pipeline.predict(forcast_data)

# set model for pipeline as OLG neural net regression model
pipeline.named_steps['estimator'].model = load_model('olg_nn_reg.h5')

# store predictions
predictions['olg'] = pipeline.predict(forcast_data)

# sum predictions and multiply by 3 (weather forcast every 3 hours), divide 1000 (use kW instead of Watts)
olg_5_day = predictions['olg'].sum() * 3 / 1000
mpls_5_day = predictions['mpls'].sum() * 3 / 1000

In [23]:
# Print results
print("5-day Solar Power Predictions:")
print(f'  Ogilvie: {olg_5_day} kW')
print(f'  Minneapolis: {mpls_5_day} kW')

5-day Solar Power Predictions:
  Ogilvie: 2008.9965 kW
  Minneapolis: 109.7003671875 kW
